# VacationPy (Part 2)
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load files
cities_file = "output_data/cities.csv"

# Read school data file and store into Pandas DataFrames
cities_df = pd.read_csv(cities_file)

cities_df.head()

,City_ID,City,Country,Latitude,Longitude,Date,Max Temperature (C),Humidity,Cloudiness,Wind Speed (km/h)
0,0,Rikitea,PF,-23.1203,-134.9692,2021-03-19 16:57:18,26.85,76,100,5.68
1,1,Port Alfred,ZA,-33.5906,26.8910,2021-03-19 16:57:56,20.03,83,98,6.30
2,2,Coos Bay,US,43.3665,-124.2179,2021-03-19 16:59:55,8.33,87,90,2.06
3,3,Tsumeb,NaN,-19.2333,17.7167,2021-03-19 16:59:55,22.73,68,74,3.01
4,4,Bredasdorp,ZA,-34.5322,20.0403,2021-03-19 16:56:57,20.00,60,100,3.60


### Humidity Heatmap

In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

#Store humidity levels
humidity = cities_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(4,17), zoom_level=1.9,layout={'width': '965px', 'height': '800px'})

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=500,
                                 point_radius=7)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='800px', width='965px'))

### Create new DataFrame fitting weather criteria

In [5]:
# Conditions
min_temp = 18
max_temp = 32
max_wind = 10
cloud_level = 0

vacation_df = cities_df.loc[(cities_df["Max Temperature (C)"]<=max_temp) 
                            & (cities_df["Max Temperature (C)"]>=min_temp)
                            & (cities_df["Cloudiness"]==cloud_level)
                            & (cities_df["Wind Speed (km/h)"]<=max_wind),:]
vacation_df.head()
 

,City_ID,City,Country,Latitude,Longitude,Date,Max Temperature (C),Humidity,Cloudiness,Wind Speed (km/h)
46,46,Chui,UY,-33.6971,-53.4616,2021-03-19 16:57:46,22.00,54,0,3.15
70,70,Eyl,SO,7.9803,49.8164,2021-03-19 17:00:12,25.68,78,0,6.86
91,91,Mar del Plata,AR,-38.0023,-57.5575,2021-03-19 16:57:21,20.56,49,0,6.17
102,102,Sur,OM,22.5667,59.5289,2021-03-19 17:00:19,26.70,40,0,2.86
124,124,Carnarvon,AU,-24.8667,113.6333,2021-03-19 16:57:15,26.00,69,0,4.63


### Hotel Map

In [6]:
# Finding nearest hotel

# geocoordinates
latitude=vacation_df["Latitude"].tolist()
longitude=vacation_df["Longitude"].tolist()
lcity=vacation_df["City"].tolist()
lcountry=vacation_df["Country"].tolist()

# set up lists to hold reponse info
hotelname = []
hotelcity = []
hotelcountry = []
lat = []
lon = []

# Loop through the list of cities and perform a request for data on each
for x,y,i,j in zip(latitude,longitude,lcity,lcountry):
    
    # Hotel search condition
    target_search = "Hotel"
    target_radius = 5000
    
    # set up a parameters dictionary
    params = {
    "location": f"{x},{y}",
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
    }

    
    # Google Place
    place_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    place_response = requests.get(place_url, params=params).json()
    
    
    try:
        hotelname.append(place_response["results"][0]["name"])
        lat.append(place_response["results"][0]["geometry"]["location"]["lat"])
        lon.append(place_response["results"][0]["geometry"]["location"]["lng"])
        hotelcity.append(i)
        hotelcountry.append(j)
        
        print(f"Coordinate ({x},{y}) found. Appending...")
    
    except:
       # Append null values
       print(f"Coordinate ({x},{y}) not found. Skipping...")
    pass

Coordinate (-33.6971,-53.4616) found. Appending...
Coordinate (7.9803,49.8164) found. Appending...
Coordinate (-38.0023,-57.5575) found. Appending...
Coordinate (22.5667,59.5289) found. Appending...
Coordinate (-24.8667,113.6333) found. Appending...
Coordinate (19.9333,-105.25) found. Appending...
Coordinate (20.930999999999997,-17.0347) found. Appending...
Coordinate (22.5667,59.5289) found. Appending...
Coordinate (-33.6971,-53.4616) found. Appending...
Coordinate (-24.8667,113.6333) found. Appending...
Coordinate (23.1225,72.0484) found. Appending...
Coordinate (-11.5714,-77.2672) found. Appending...
Coordinate (21.6422,69.6093) found. Appending...
Coordinate (18.2164,42.5053) found. Appending...
Coordinate (-38.5473,-58.7368) found. Appending...
Coordinate (-38.0023,-57.5575) found. Appending...
Coordinate (17.0151,54.0924) found. Appending...
Coordinate (17.0151,54.0924) found. Appending...
Coordinate (-43.3002,-65.1023) found. Appending...
Coordinate (25.2088,64.6357) found. Appe

In [7]:
# create a data frame of hotels
hotel_df = pd.DataFrame({"Hotel Name": hotelname,
                            "Lat": lat,
                            "Lng": lon,
                            "City": hotelcity,
                            "Country": hotelcountry
                            })

hotel_df.head()

,Hotel Name,Lat,Lng,City,Country
0,Descanso al Paso Chuy,-33.700201,-53.452834,Chui,UY
1,DAAWAD HOTEL EYL,7.983365,49.817706,Eyl,SO
2,NH Gran Hotel Provincial,-38.006511,-57.541344,Mar del Plata,AR
3,Sur Plaza Hotel,22.566435,59.495858,Sur,OM
4,Carnarvon Sea Change Apartments,-24.884147,113.655056,Carnarvon,AU


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
#Locations for heatmap
hlocations = cities_df[["Latitude", "Longitude"]]

#Store humidity levels
humidity = cities_df["Humidity"]

# Plot standard map
fig = gmaps.figure(center=(4,17), zoom_level=1.9,layout={'width': '965px', 'height': '800px'})

# Create location marker
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Create heat layer
heat_layer = gmaps.heatmap_layer(hlocations, weights=humidity, 
                                 dissipating=False, max_intensity=500,
                                 point_radius=7)

# Add layer
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='800px', width='965px'))